In [1]:
import sys

sys.path.append('..')

In [2]:
from VisionSystem import VisionSystem, VideoStream, VisualObject
from VisionSystem.DetectionModel import ThreshBlob
from DisplayPane import DisplayPane
from KickerSystem import KickerSystem
from DriveSystem import DriveSystem
from NavigationSystem import NavigationSystem

before {}
after {13: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 10', <gpiozero.LED object on pin GPIO13, active_high=True, is_active=False>), 26: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 12', <gpiozero.LED object on pin GPIO26, active_high=True, is_active=False>)}
before {13: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 10', <gpiozero.LED object on pin GPIO13, active_high=True, is_active=False>), 26: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 12', <gpiozero.LED object on pin GPIO26, active_high=True, is_active=False>)}
after {13: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 10', <gpiozero.LED object on pin GPIO13, active_high=True, is_active=False>), 26: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 12', <gpiozero.LED object on pin GPIO26, active_high=True, is_active=False>), 5: ('/home/pi/SoccerRobot/DriveSystem/DriveSystem.py, line 10', <gpiozero.LED object on pin GPIO5, active_high=True, is_active=False>), 12: 

In [3]:
drive = DriveSystem()
kicker = KickerSystem()
kicker.setup() # dont ask

In [4]:
def load_or_create_new_threshblob(path):
    try:
        model = ThreshBlob.load(path)
        print("Loaded " + path)
    except Exception:
        model = ThreshBlob()
    finally:
        return model
    

def setup_vision_system(resolution):
    objects_to_size_and_result_limit = [
        ("ball", (0.043, 0.043, 0.043), 1),
        ("blue_goal", (0, 0, 0.1), 1),
        ("obstacle", (0.18, 0.18, 0.2), None),
        # 30 centimetres long, 10 cm high? i guess
        ("yellow_goal", (0, 0, 0.1), 1)
    ]

    return VisionSystem(
        resolution=resolution,
        objects_to_track={
            name: VisualObject(
                real_size=size,
                detection_model=load_or_create_new_threshblob(f"../models/{name}.threshblob.pkl"),
                result_limit=result_limit
            ) for name, size, result_limit in objects_to_size_and_result_limit
        }
    )

video = VideoStream(downsample_scale=5, crop=((0.13, 0), (.9, 1)))
vision = setup_vision_system(video.resolution)
nav_system = NavigationSystem(vision, drive, kicker)

Loaded ../models/ball.threshblob.pkl
Loaded ../models/blue_goal.threshblob.pkl
Loaded ../models/obstacle.threshblob.pkl
Loaded ../models/yellow_goal.threshblob.pkl


In [5]:
# from DisplayPane.Interactor.VisionSystemTuner import VisionSystemTuner

# DisplayPane(vision_system=vision, video_stream=video, interactors=[VisionSystemTuner(vision)])

In [6]:
# RUN THIS CELL TO SAVE THE MODELs tinkered with
for name, obj in vision.objects_to_track.items():
    obj.detection_model.save(f"../models/{name}.threshblob.pkl")

In [7]:
# # def live_test():
# for frame in video_stream:
#     vision_system.update_with_frame(frame)
#     nav_system.update()

#     time.sleep(0.2)
#     drive_system.set_desired_motion(0, 0, 0)

In [8]:
# for _ in live_test():
#     pass

In [9]:
# import ipywidgets as ipy

# this_test = live_test()

# step_button = ipy.Button(
#     description='Next Step'
# )

# step_button.on_click(lambda _: next(this_test))
# step_button

In [10]:
def get_vision_results_vrep_format():
    objs = vision.objects_to_track # for shorthand

    def vrep_format(bearings_distances, multi=False):
        if any(bearings_distances):
            if multi:
                return bearings_distances[::-1]
            else:
                return bearings_distances[0][::-1]
        else:
            return None

    return (
        vrep_format(objs["ball"].bearings_distances),
        vrep_format(objs["blue_goal"].bearings_distances),
        vrep_format(objs["yellow_goal"].bearings_distances),
        vrep_format(objs["obstacle"].bearings_distances, multi=True),
    )

In [11]:
# import timeit

# frames = iter(video_stream)

# def frame_rate():
#     vision_system.update_with_frame(next(frames))

# n_frames_to_test = 100
# hz = n_frames_to_test / timeit.timeit(frame_rate, number=n_frames_to_test)


In [12]:
# hz

In [13]:
# vision_system.update_with_frame(next(frames))

In [14]:
drive.set_desired_motion(0, 0, 0)

In [ ]:
from math import *
from time import sleep

nav_system.state = None
for frame in video:
    vision.update_with_frame(frame)
    ball_br, blue_br, yellow_br, obstacle_brs = get_vision_results_vrep_format()
    nav_system.update(ball_br, yellow_br, blue_br, obstacle_brs)

../VisionSystem/DetectionModel/ThreshBlob/ThreshBlob.py:22: RuntimeWarning: overflow encountered in exp
  return (0.1407675 * np.exp(0.85425173 * img_dist / RESCALE / frac_of_full_res) + 0.79365708) * DIST_PER_POINT
../VisionSystem/DetectionModel/DetectionResult.py:29: RuntimeWarning: overflow encountered in long_scalars
  return 0.5*np.abs(np.dot(x, np.roll(y, 1))-np.dot(y, np.roll(x, 1)))


Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball with goal
Translating to align ball wi

Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goa

Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goal
hello
Translating to align ball with goa

In [ ]:
from DisplayPane import DisplayPane
from DisplayPane.Interactor.VisionSystemTuner import VisionSystemTuner

DisplayPane(video_stream=video, vision_system=vision, interactors=[VisionSystemTuner(vision)])

In [ ]:
drive.set_desired_motion(0, 0, 0)

In [ ]:
# RUN THIS CELL TO SAVE THE MODELs tinkered with
for name, obj in vision.objects_to_track.items():
    obj.detection_model.save("../models/" + name + ".threshblob.pkl")